<a href="https://colab.research.google.com/github/MehmetCanYildirim/Projects/blob/main/%5BAAN%5D_EEG_Emotion_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
from google.colab import files


uploaded = files.upload()

Saving emotions.csv to emotions.csv


In [ ]:
data = pd.read_csv('emotions.csv')
data

In [ ]:
data['label'].value_counts()

In [ ]:
label_mapping = {'NEGATIVE':0, 'NEUTRAL':1 , 'POSITIVE':2}

In [ ]:
data['label'].replace(label_mapping)

In [ ]:
def preprocessing(df):
    df = df.copy()
    df['label'] = df['label'].replace(label_mapping)
    y = df['label'].copy()
    X = df.drop('label', axis=1).copy()

    X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=123)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocessing(data)

In [ ]:
X_test.shape

NameError: ignored

In [ ]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)

outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs= inputs, outputs=outputs)

print(model.summary())

In [ ]:
model.compile(optimizer='adam',
             loss ='sparse_categorical_crossentropy',
             metrics=['accuracy']
             )

history = model.fit(X_train,
                    y_train,
                    batch_size=32,
                    epochs=50,
                    validation_split=0.2)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

In [ ]:
yprediction = np.array(list(map(lambda a: np.argmax(a), model.predict(X_test))))


con_mat = confusion_matrix(y_test, yprediction)
class_rep = classification_report(y_test, yprediction, target_names = label_mapping.keys())
print(class_rep)

plt.figure(figsize=(10,8))
sns.heatmap(con_mat, annot = True, vmin=0, cbar = True, fmt ='g', cmap='Reds')
plt.xlabel('Prediction')
plt.ylabel('Real')
plt.title("Prediction vs Real Value")
plt.show()